# Feature extraction by model on Places365 for EmoSet

In [1]:
import torch
from torch.autograd import Variable as V
import torchvision.models as models
from torchvision import transforms as trn
from torch.nn import functional as F
import os
from PIL import Image
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
print(device)

# th architecture to use
arch = 'resnet18'

# load the pre-trained weights
model_file = 'resnet18_places365.pth.tar'

model = models.__dict__[arch](num_classes=365)
checkpoint = torch.load(model_file, map_location=lambda storage, loc: storage)
state_dict = {str.replace(k,'module.',''): v for k,v in checkpoint['state_dict'].items()}
model.load_state_dict(state_dict)

cuda


<All keys matched successfully>

In [2]:
imgs_list = []
labels_list = []

In [3]:
# Чтение файла и заполнение списка img_list
with open("img_paths_list.txt", "r") as file:
    for line in file:
        # Удаление символа новой строки и разделение строки на пути к изображениям
        paths = line.strip().split()

        # Добавление путей к изображениям в список
        imgs_list.extend(paths)

# Вывод списка путей к изображениям
print(imgs_list[0:5])

['image/amusement/amusement_08258.jpg', 'image/excitement/excitement_17835.jpg', 'image/excitement/excitement_09581.jpg', 'image/anger/anger_01725.jpg', 'image/fear/fear_10809.jpg']


In [4]:
# Чтение файла и заполнение списка labels_list
with open("labels_list.txt", "r") as file:
    for line in file:
        # Преобразование строки в целое число и добавление его в список
        label = int(line.strip())
        labels_list.append(label)

# Вывод списка меток
print(labels_list[0:5])

[0, 3, 3, 4, 6]


In [5]:
print(len(imgs_list))
print(len(labels_list))

118102
118102


In [6]:
x_FE = []

In [7]:
print(len(x_FE))
print(imgs_list[0:5])

0
['image/amusement/amusement_08258.jpg', 'image/excitement/excitement_17835.jpg', 'image/excitement/excitement_09581.jpg', 'image/anger/anger_01725.jpg', 'image/fear/fear_10809.jpg']


##### Ниже представлен демонстрационный вариант запуска процесса извлечения признаков. Признаки из каждого изображения уже были извлечены ранее и записаны в предложенный файл, из которого ниже происходит чтение и заполнение списка X

In [8]:
model.fc =torch.nn.Identity()
model=model.to(device)
model=model.eval()

In [9]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [11]:
# load the image transformer
centre_crop = trn.Compose([
        trn.Resize((256,256)),
        trn.CenterCrop(224),
        trn.ToTensor(),
        trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [28]:
for path in imgs_list:
    try:
        img = Image.open(f'C:\EmoSet\EmoSet-118K/{path}')
        input_img = V(centre_crop(img).unsqueeze(0))
        input_img = input_img.to(device)
        features = model(input_img)
        features=features.data.cpu().numpy()
        x_FE.append(features)
    except FileNotFoundError:
    # обработка ошибки, например, вывод сообщения или просто пропуск
        print("Файл не найден. Продолжаем выполнение программы.")

Файл не найден. Продолжаем выполнение программы.


In [29]:
print(len(x_FE))
print(len(x_FE[0]))
print(x_FE[0].shape)

118101
1
(1, 512)


In [31]:
print(x_FE[0])
print(x_FE[1])

[[6.71541333e-01 1.39890516e+00 1.85912892e-01 2.90061116e-01
  3.77153695e-01 2.16124788e-01 2.14264765e-01 1.63691986e+00
  1.30857259e-01 1.00815582e+00 6.62031770e-01 4.92778778e-01
  1.20593333e+00 3.01605344e-01 9.53539431e-01 1.87279224e-01
  2.16938066e+00 9.40227658e-02 5.49742162e-01 1.80491149e-01
  9.82153833e-01 5.93475044e-01 1.03124723e-01 4.38776851e-01
  4.52014923e-01 2.21265405e-01 6.82294011e-01 7.39441395e-01
  6.15641356e-01 2.81304061e-01 1.65271986e+00 6.57850385e-01
  2.36199045e+00 2.62140445e-02 2.48465702e-01 1.31880278e-02
  1.25684261e-01 5.41759729e-01 3.22544962e-01 7.78905094e-01
  2.78291464e+00 8.17420781e-01 9.82392952e-02 1.09851444e+00
  9.38858151e-01 2.15977311e-01 3.82732689e-01 3.47302020e-01
  1.49359667e+00 1.31296003e+00 1.47364840e-01 4.29573864e-01
  2.58094341e-01 2.08044514e-01 1.01302683e+00 6.41855180e-01
  2.06947112e+00 9.42091178e-03 5.01396477e-01 8.26078773e-01
  1.48548639e+00 5.49800456e-01 2.02835426e-01 1.03400111e+00
  1.6893

## Запись extracted features в файл

In [ ]:
output_file = "x_FE_places365.txt"

with open(output_file, "w") as file:
    for arr in x_FE:
        for a in arr:
            file.write('[')
            for elem in a:
                file.write(str(elem) + ", ")
            file.write(']')
        file.write("\n")

# Начало экспериментов отсюда

## Чтение файла x_FE_HSEmotion в другой массив

In [13]:
import numpy as np
x_FE_v1 = []

# Чтение файла и заполнение списка X
with open("x_FE_places365.txt", "r") as file:
    for line in file:
        # Преобразование строки в список чисел
        line = line.strip().strip('[]').split(', ')
#         print(line)
        arr = np.array([float(num) for num in line if num != ''])

        # Решейп массива к размерности (1, 1280)
        arr = arr.reshape(1, -1)

        # Добавление массива в список
        x_FE_v1.append(arr)

## Заполнение массивов X и Y элементами подходящей размерности

In [14]:
X = []

In [15]:
for item in x_FE_v1:
    x = torch.tensor(item)
    x = x.view(1, 1, 512).type(torch.float32)
    x = x.repeat(3, 1, 1)
    x.to(device)
    X.append(x)

In [16]:
Y = []

In [17]:
# изменение элементов массива labels_list (y) чтобы подать модели на вход на обучение
for item in labels_list:
    y = torch.tensor(item)
#     y = F.one_hot(y, 8).float()
    y.to(device)
    Y.append(y)
#     print(y)

In [18]:
del Y[91900]

## Деление на train, valid наборы

In [47]:
import sklearn
from sklearn.model_selection import train_test_split

In [48]:
# Разделяем данные на train/validation sets
x_train, x_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.2, random_state=42)

## Формирование train, valid датасетов из Dataset и Dataloader

In [49]:
from torch.utils.data import Dataset, DataLoader

In [50]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [51]:
dataset = CustomDataset(x_train, y_train)

In [52]:
batch_size = 8
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [53]:
for sample in train_dataloader:
    print("Длина sample") # здесь длина такая, так как у нас в сэмпле содержится x и y
    print(len(sample))
    print("Пример sample:\n")
    print(sample)

    print("Длина фичей или меток в одном sample должна быть:\n")
    print(len(sample[0]))
    print("Длина фичей или меток в одном sample должна быть:\n")
    print(sample[0][0].shape)
    break

Длина sample
2
Пример sample:

[tensor([[[[7.1050e-01, 1.4135e+00, 1.6840e+00,  ..., 6.2636e-02,
           8.1859e-02, 1.0066e+00]],

         [[7.1050e-01, 1.4135e+00, 1.6840e+00,  ..., 6.2636e-02,
           8.1859e-02, 1.0066e+00]],

         [[7.1050e-01, 1.4135e+00, 1.6840e+00,  ..., 6.2636e-02,
           8.1859e-02, 1.0066e+00]]],


        [[[1.6395e-01, 1.7349e-01, 7.7421e-01,  ..., 6.0810e-01,
           1.3330e-01, 3.2869e-01]],

         [[1.6395e-01, 1.7349e-01, 7.7421e-01,  ..., 6.0810e-01,
           1.3330e-01, 3.2869e-01]],

         [[1.6395e-01, 1.7349e-01, 7.7421e-01,  ..., 6.0810e-01,
           1.3330e-01, 3.2869e-01]]],


        [[[2.8563e-02, 2.2261e-01, 8.9990e-01,  ..., 6.2367e-01,
           3.8143e-01, 4.6105e-02]],

         [[2.8563e-02, 2.2261e-01, 8.9990e-01,  ..., 6.2367e-01,
           3.8143e-01, 4.6105e-02]],

         [[2.8563e-02, 2.2261e-01, 8.9990e-01,  ..., 6.2367e-01,
           3.8143e-01, 4.6105e-02]]],


        ...,


        [[[5.8266e-0

In [54]:
dataset_v = CustomDataset(x_valid, y_valid)

In [55]:
# DataLoader для пакетной загрузки данных
batch_size = 8
valid_dataloader = DataLoader(dataset_v, batch_size=batch_size, shuffle=True)

In [56]:
# Пример использования DataLoader
for sample in valid_dataloader:
    print("Длина sample")
    print(len(sample))
    print("Пример sample:\n")
    print(sample)
    
    print("Длина фичей или меток в одном sample должна быть:\n")
    print(len(sample[0]))
    print("Длина фичей или меток в одном sample должна быть:\n")
    print(sample[0][0].shape)
    break

Длина sample
2
Пример sample:

[tensor([[[[0.0000, 0.0000, 0.0107,  ..., 0.5793, 0.7542, 0.1619]],

         [[0.0000, 0.0000, 0.0107,  ..., 0.5793, 0.7542, 0.1619]],

         [[0.0000, 0.0000, 0.0107,  ..., 0.5793, 0.7542, 0.1619]]],


        [[[0.0078, 0.4962, 0.3490,  ..., 0.8653, 0.3264, 0.0106]],

         [[0.0078, 0.4962, 0.3490,  ..., 0.8653, 0.3264, 0.0106]],

         [[0.0078, 0.4962, 0.3490,  ..., 0.8653, 0.3264, 0.0106]]],


        [[[0.1328, 0.0865, 0.5412,  ..., 0.0154, 0.4775, 0.8958]],

         [[0.1328, 0.0865, 0.5412,  ..., 0.0154, 0.4775, 0.8958]],

         [[0.1328, 0.0865, 0.5412,  ..., 0.0154, 0.4775, 0.8958]]],


        ...,


        [[[0.0000, 0.0179, 0.4027,  ..., 0.5101, 0.0194, 0.1280]],

         [[0.0000, 0.0179, 0.4027,  ..., 0.5101, 0.0194, 0.1280]],

         [[0.0000, 0.0179, 0.4027,  ..., 0.5101, 0.0194, 0.1280]]],


        [[[0.2427, 1.5560, 0.4238,  ..., 0.5001, 0.1669, 0.4786]],

         [[0.2427, 1.5560, 0.4238,  ..., 0.5001, 0.1669, 0.47

In [57]:
print(len(train_dataloader))
print(len(valid_dataloader))

11810
2953


In [58]:
print(len(x_train))

94480


In [59]:
print(len(x_valid))

23621


In [60]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [61]:
import copy
def test(val_loader,model,device,criterion):
    model.eval()
    epoch_val_accuracy,epoch_val_loss = 0,0 
    with torch.no_grad():
        for sample in val_loader:
            X, y = sample[0], sample[1]
            X = X.to(device)
            y = y.to(device)
            
            val_output = model(X)
            if criterion is None:
                val_loss = F.nll_loss(val_output, y, reduction='sum').item()
            else:
                val_loss = criterion(val_output, y)
            
            acc = (val_output.argmax(dim=1) == y).float().mean()
            epoch_val_accuracy += acc 
            epoch_val_loss += val_loss
    return epoch_val_accuracy/ len(val_loader),epoch_val_loss/ len(val_loader)

def train(train_loader,val_loader,model,device,n_epochs, optimizer,criterion,scheduler):
    train_accuracies,val_accuracies=[],[]
    best_acc=0
    best_model=None
    for epoch in range(n_epochs):
        epoch_loss = 0
        epoch_accuracy = 0
        model.train()
#         for sample in tqdm(train_loader):
        for sample in train_loader:
            X, y = sample[0], sample[1]
            X = X.to(device)
#             print(X)
            y = y.to(device)
            output = model(X)

            loss = criterion(output, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            acc = (output.argmax(dim=1) == y).float().mean()
            
            epoch_accuracy += acc / len(train_loader)
            epoch_loss += loss / len(train_loader)

        epoch_val_accuracy,epoch_val_loss = test(val_loader,model,device,criterion)
#         print(
#             f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
#         )
        train_accuracies.append(epoch_accuracy)
        val_accuracies.append(epoch_val_accuracy)
        if best_acc<epoch_val_accuracy:
            best_acc=epoch_val_accuracy
            best_model=copy.deepcopy(model.state_dict())
        if scheduler is not None:
            scheduler.step()
    
    if best_model is not None:
        model.load_state_dict(best_model)
        print(f"Best acc:{best_acc}")
        epoch_val_accuracy,epoch_val_loss = test(val_loader,model,device,criterion)
        print(
            f"val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
        )
    else:
        print(f"No best model Best acc:{best_acc}")
    return best_model,train_accuracies,val_accuracies

## Загрузка EfficientNet и настройка

### Transfer learning

In [62]:
efficientnet = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_b0', pretrained=True)
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_convnets_processing_utils')

print(efficientnet)

EfficientNet(
  (stem): Sequential(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (activation): SiLU(inplace=True)
  )
  (layers): Sequential(
    (0): Sequential(
      (block0): MBConvBlock(
        (depsep): Sequential(
          (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (se): SequentialSqueezeAndExcitation(
          (squeeze): Linear(in_features=32, out_features=8, bias=True)
          (expand): Linear(in_features=8, out_features=32, bias=True)
          (activation): SiLU(inplace=True)
          (sigmoid): Sigmoid()
          (mul_a_quantizer): Identity()
          (mul_b_quantizer): Identity()
        )
      

Using cache found in C:\Users\Юлия/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub
Using cache found in C:\Users\Юлия/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub


In [63]:
num_classes=8

classifier = nn.Sequential(
    nn.AdaptiveAvgPool2d(output_size=1),
    nn.Flatten(),
    nn.Dropout(p=0.2, inplace=False),
    nn.Linear(1280, 8)
)

print("Num parameters FC")
print(count_parameters(classifier))

print("Num parameters ALL")
print(count_parameters(efficientnet))

efficientnet.classifier = classifier

Num parameters FC
10248
Num parameters ALL
5288548


In [64]:
print(efficientnet)

EfficientNet(
  (stem): Sequential(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (activation): SiLU(inplace=True)
  )
  (layers): Sequential(
    (0): Sequential(
      (block0): MBConvBlock(
        (depsep): Sequential(
          (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (se): SequentialSqueezeAndExcitation(
          (squeeze): Linear(in_features=32, out_features=8, bias=True)
          (expand): Linear(in_features=8, out_features=32, bias=True)
          (activation): SiLU(inplace=True)
          (sigmoid): Sigmoid()
          (mul_a_quantizer): Identity()
          (mul_b_quantizer): Identity()
        )
      

In [65]:
for param in efficientnet.parameters():
    param.requires_grad = False

In [66]:
for param in efficientnet.classifier.parameters():
    param.requires_grad = True

In [67]:
print("Num parameters ALL")
print(count_parameters(efficientnet))

Num parameters ALL
10248


In [45]:
n_epochs = 15
efficientnet=efficientnet.to(device)
optimizer = torch.optim.Adam(efficientnet.classifier.parameters(), lr=0.001, betas=(0.9, 0.999))
# optimizer = optim.SGD(filter(lambda p: p.requires_grad, cnn_model.parameters()), lr=0.1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.8)
# scheduler нужен для динамического регулирования learning rate (скорости обучения) в процессе обучения.
criterion=nn.CrossEntropyLoss()

In [68]:
best_model,train_accuracies,val_accuracies=train(train_dataloader,valid_dataloader,efficientnet,device,n_epochs, optimizer,criterion,scheduler)

Best acc:0.4215712785720825
val_loss : 1.2308 - val_acc: 0.4216


In [69]:
model_path = "scene_efficientnet_tl_final.pth"
torch.save(best_model, model_path)

### Fine tuning

In [70]:
efficientnet = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_b0', pretrained=True)
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_convnets_processing_utils')

print(efficientnet)

EfficientNet(
  (stem): Sequential(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (activation): SiLU(inplace=True)
  )
  (layers): Sequential(
    (0): Sequential(
      (block0): MBConvBlock(
        (depsep): Sequential(
          (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (se): SequentialSqueezeAndExcitation(
          (squeeze): Linear(in_features=32, out_features=8, bias=True)
          (expand): Linear(in_features=8, out_features=32, bias=True)
          (activation): SiLU(inplace=True)
          (sigmoid): Sigmoid()
          (mul_a_quantizer): Identity()
          (mul_b_quantizer): Identity()
        )
      

Using cache found in C:\Users\Юлия/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub
Using cache found in C:\Users\Юлия/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub


In [71]:
num_classes=8
classifier = nn.Sequential(
    nn.AdaptiveAvgPool2d(output_size=1),
    nn.Flatten(),
    nn.Dropout(p=0.2, inplace=False),
    nn.Linear(1280, 8)
)

print("Num parameters FC")
print(count_parameters(classifier))

print("Num parameters ALL")
print(count_parameters(efficientnet))

efficientnet.classifier = classifier

Num parameters FC
10248
Num parameters ALL
5288548


In [72]:
print(efficientnet)

EfficientNet(
  (stem): Sequential(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (activation): SiLU(inplace=True)
  )
  (layers): Sequential(
    (0): Sequential(
      (block0): MBConvBlock(
        (depsep): Sequential(
          (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (se): SequentialSqueezeAndExcitation(
          (squeeze): Linear(in_features=32, out_features=8, bias=True)
          (expand): Linear(in_features=8, out_features=32, bias=True)
          (activation): SiLU(inplace=True)
          (sigmoid): Sigmoid()
          (mul_a_quantizer): Identity()
          (mul_b_quantizer): Identity()
        )
      

In [73]:
for param in efficientnet.parameters():
    param.requires_grad = True

In [74]:
print("Num parameters ALL")
print(count_parameters(efficientnet))

Num parameters ALL
4017796


In [75]:
n_epochs = 15
efficientnet=efficientnet.to(device)
optimizer = torch.optim.Adam(efficientnet.parameters(), lr=0.001, betas=(0.9, 0.999))
# optimizer = optim.SGD(filter(lambda p: p.requires_grad, cnn_model.parameters()), lr=0.1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.8)
# scheduler нужен для динамического регулирования learning rate (скорости обучения) в процессе обучения.
criterion=nn.CrossEntropyLoss()

In [76]:
best_model,train_accuracies,val_accuracies=train(train_dataloader,valid_dataloader,efficientnet,device,n_epochs, optimizer,criterion,scheduler)

Best acc:0.8047226360321045
val_loss : 0.6479 - val_acc: 0.8047


In [78]:
torch.save(efficientnet, "scene_efficientnet_ft_final.pth")

## Загрузка ResNet-50 и настройка

### Transfer learning

In [79]:
from torchvision import models
cnn_model = models.resnet50(pretrained=True)
# cnn_model = models.resnet18(weights="IMAGENET1K_V1")
print(cnn_model)

C:\anaconda3\envs\pytorch_gpu\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\anaconda3\envs\pytorch_gpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [80]:
num_classes=8

fc = nn.Linear(2048, 8, bias=True)

print("Num parameters FC")
print(count_parameters(fc))

print("Num parameters ALL")
print(count_parameters(cnn_model))

cnn_model.fc = fc

Num parameters FC
16392
Num parameters ALL
25557032


In [81]:
print(cnn_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [82]:
for param in cnn_model.parameters():
    param.requires_grad = False

In [83]:
for param in cnn_model.fc.parameters():
    param.requires_grad = True

In [84]:
print("Num parameters ALL")
print(count_parameters(cnn_model))

Num parameters ALL
16392


In [85]:
from torch.optim.lr_scheduler import StepLR

In [86]:
n_epochs = 15
cnn_model=cnn_model.to(device)
optimizer = torch.optim.Adam(cnn_model.fc.parameters(), lr=0.001, betas=(0.9, 0.999))
# optimizer = optim.SGD(filter(lambda p: p.requires_grad, cnn_model.parameters()), lr=0.1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.8)
# scheduler нужен для динамического регулирования learning rate (скорости обучения) в процессе обучения.
criterion=nn.CrossEntropyLoss()

In [88]:
best_model,train_accuracies,val_accuracies=train(train_dataloader,valid_dataloader,cnn_model,device,n_epochs, optimizer,criterion,scheduler)

Best acc:0.47904028509140015
val_loss : 1.0823 - val_acc: 0.4790


In [89]:
model_path = "scene_resnet50_tl_final.pth"
torch.save(best_model, model_path)

### Fine tuning

In [90]:
from torchvision import models
cnn_model = models.resnet50(pretrained=True)
# cnn_model = models.resnet18(weights="IMAGENET1K_V1")
print(cnn_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [91]:
num_classes=8
fc = nn.Linear(2048, 8, bias=True)

print("Num parameters FC")
print(count_parameters(fc))

print("Num parameters ALL")
print(count_parameters(cnn_model))

cnn_model.fc = fc

Num parameters FC
16392
Num parameters ALL
25557032


In [92]:
print(cnn_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [93]:
for param in cnn_model.parameters():
    param.requires_grad = True

In [94]:
print("Num parameters ALL")
print(count_parameters(cnn_model))

Num parameters ALL
23524424


In [95]:
n_epochs = 15
cnn_model=cnn_model.to(device)
optimizer = torch.optim.Adam(cnn_model.parameters(), lr=0.001, betas=(0.9, 0.999))
# optimizer = optim.SGD(filter(lambda p: p.requires_grad, cnn_model.parameters()), lr=0.1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.8)
# scheduler нужен для динамического регулирования learning rate (скорости обучения) в процессе обучения.
criterion=nn.CrossEntropyLoss()

In [96]:
best_model,train_accuracies,val_accuracies=train(train_dataloader,valid_dataloader,cnn_model,device,n_epochs, optimizer,criterion,scheduler)

Best acc:0.7653931782091730993
val_loss : 0.6773 - val_acc: 0.7654


In [97]:
torch.save(cnn_model, "scene_resnet50_ft_final.pth")

## Загрузка ResNet18 и настройка

### Transfer learning

In [98]:
from torchvision import models
# cnn_model = models.mobilenet_v2(pretrained=True)
cnn_model = models.resnet18(weights="IMAGENET1K_V1")
print(cnn_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [99]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [100]:
num_classes=8

fc = nn.Sequential(
    nn.Linear(512, num_classes)
)

print("Num parameters FC")
print(count_parameters(fc))

print("Num parameters ALL")
print(count_parameters(cnn_model))

cnn_model.fc = fc

Num parameters FC
4104
Num parameters ALL
11689512


In [101]:
print(cnn_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [102]:
for param in cnn_model.parameters():
    param.requires_grad = False

In [103]:
for param in cnn_model.fc.parameters():
    param.requires_grad = True

In [104]:
print("Num parameters ALL")
print(count_parameters(cnn_model))

Num parameters ALL
4104


In [105]:
from torch.optim.lr_scheduler import StepLR

In [106]:
n_epochs = 15
cnn_model=cnn_model.to(device)
optimizer = torch.optim.Adam(cnn_model.fc.parameters(), lr=0.001, betas=(0.9, 0.999))
# optimizer = optim.SGD(filter(lambda p: p.requires_grad, cnn_model.parameters()), lr=0.1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.8)
# scheduler нужен для динамического регулирования learning rate (скорости обучения) в процессе обучения.
criterion=nn.CrossEntropyLoss()

In [108]:
best_model,train_accuracies,val_accuracies=train(train_dataloader,valid_dataloader,cnn_model,device,n_epochs, optimizer,criterion,scheduler)

Best acc:0.3703369355201721
val_loss : 1.4014 - val_acc: 0.3703


In [109]:
model_path = "scene_resnet18_tl_final.pth"
torch.save(best_model, model_path)

### Fine tuning

In [110]:
from torchvision import models
# cnn_model = models.mobilenet_v2(pretrained=True)
cnn_model = models.resnet18(weights="IMAGENET1K_V1")
print(cnn_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [111]:
num_classes=8


fc = nn.Sequential(
    nn.Linear(512, num_classes)
)

print("Num parameters FC")
print(count_parameters(fc))

print("Num parameters ALL")
print(count_parameters(cnn_model))

cnn_model.fc = fc

Num parameters FC
4104
Num parameters ALL
11689512


In [112]:
print(cnn_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [113]:
for param in cnn_model.parameters():
    param.requires_grad = True

In [114]:
print("Num parameters ALL")
print(count_parameters(cnn_model))

Num parameters ALL
11180616


In [115]:
n_epochs = 15
cnn_model=cnn_model.to(device)
optimizer = torch.optim.Adam(cnn_model.parameters(), lr=0.001, betas=(0.9, 0.999))
# optimizer = optim.SGD(filter(lambda p: p.requires_grad, cnn_model.parameters()), lr=0.1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.8)
# scheduler нужен для динамического регулирования learning rate (скорости обучения) в процессе обучения.
criterion=nn.CrossEntropyLoss()

In [116]:
best_model,train_accuracies,val_accuracies=train(train_dataloader,valid_dataloader,cnn_model,device,n_epochs, optimizer,criterion,scheduler)

Best acc:0.7225380516052246
val_loss : 0.7178 - val_acc: 0.7225


In [117]:
model_path = "scene_resnet18_ft_final.pth"

In [118]:
torch.save(cnn_model, model_path)

## Загрузка MobileNet и настройка

### Transfer learning

In [123]:
from torchvision import models
cnn_model = models.mobilenet_v2(pretrained=True)
# cnn_model = models.resnet18(weights="IMAGENET1K_V1")
print(cnn_model)

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [120]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [125]:
num_classes=8
classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=False),
    nn.Linear(1280, 8, bias=True)
)

print("Num parameters FC")
print(count_parameters(fc))

print("Num parameters ALL")
print(count_parameters(cnn_model))

cnn_model.classifier = classifier

Num parameters FC
4104
Num parameters ALL
3504872


In [126]:
print(cnn_model)

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [127]:
for param in cnn_model.parameters():
    param.requires_grad = False

In [129]:
for param in cnn_model.classifier.parameters():
    param.requires_grad = True

In [130]:
print("Num parameters ALL")
print(count_parameters(cnn_model))

Num parameters ALL
10248


In [131]:
from torch.optim.lr_scheduler import StepLR

In [132]:
n_epochs = 15
cnn_model=cnn_model.to(device)
optimizer = torch.optim.Adam(cnn_model.classifier.parameters(), lr=0.001, betas=(0.9, 0.999))
# optimizer = optim.SGD(filter(lambda p: p.requires_grad, cnn_model.parameters()), lr=0.1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.8)
# scheduler нужен для динамического регулирования learning rate (скорости обучения) в процессе обучения.
criterion=nn.CrossEntropyLoss()

In [133]:
best_model,train_accuracies,val_accuracies=train(train_dataloader,valid_dataloader,cnn_model,device,n_epochs, optimizer,criterion,scheduler)

Best acc:0.4101366219839912
val_loss : 1.2751 - val_acc: 0.4101


In [134]:
model_path = "scene_mobilenet_tl_final.pth"
torch.save(best_model, model_path)

### Fine tuning

In [136]:
from torchvision import models
cnn_model = models.mobilenet_v2(pretrained=True)
# cnn_model = models.resnet18(weights="IMAGENET1K_V1")
print(cnn_model)

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [137]:
num_classes=8
classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=False),
    nn.Linear(1280, 8, bias=True)
)

print("Num parameters FC")
print(count_parameters(fc))

print("Num parameters ALL")
print(count_parameters(cnn_model))

cnn_model.classifier = classifier

Num parameters FC
4104
Num parameters ALL
3504872


In [138]:
print(cnn_model)

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [139]:
for param in cnn_model.parameters():
    param.requires_grad = True

In [140]:
print("Num parameters ALL")
print(count_parameters(cnn_model))

Num parameters ALL
2234120


In [141]:
n_epochs = 15
cnn_model=cnn_model.to(device)
optimizer = torch.optim.Adam(cnn_model.parameters(), lr=0.001, betas=(0.9, 0.999))
# optimizer = optim.SGD(filter(lambda p: p.requires_grad, cnn_model.parameters()), lr=0.1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.8)
# scheduler нужен для динамического регулирования learning rate (скорости обучения) в процессе обучения.
criterion=nn.CrossEntropyLoss()

In [142]:
best_model,train_accuracies,val_accuracies=train(train_dataloader,valid_dataloader,cnn_model,device,n_epochs, optimizer,criterion,scheduler)

Best acc:0.69975698346617032
val_loss : 0.7601 - val_acc: 0.6998


In [143]:
model_path = "scene_mobilenet_ft_final.pth"

In [144]:
torch.save(cnn_model, model_path)